## 1. Realizamos el login en la API REST

In [1]:
import logging
from json import load
# Importing models and REST client class from Professional Edition version
from tb_rest_client.rest_client_pe import *
from tb_rest_client.rest import ApiException


logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(module)s - %(lineno)d - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')


# ThingsBoard REST API URL
url = "https://development.akiot.es/"

# Default Tenant Administrator credentials
username = "tenant@akiot.es"
password = "tenant"


## 2. Nos autenticamos y obtenemos el actual usuario

In [2]:
with RestClientPE(base_url=url) as rest_client:
    try:
        # Auth with credentials
        rest_client.login(username=username, password=password)

        # Getting current user
        current_user = rest_client.get_user()
        logging.info('Current user:\n%r\n', current_user)

    except ApiException as e:
            logging.exception(e)

2025-11-05 16:38:14 - INFO - 470615400 - 8 - Current user:
{'additional_info': {'homeDashboardHideToolbar': True,
                     'lastLoginTs': None,
                     'userActivated': True,
                     'userCredentialsEnabled': True},
 'authority': 'TENANT_ADMIN',
 'created_time': 1760987466533,
 'custom_menu_id': None,
 'customer_id': {'entity_type': 'CUSTOMER',
                 'id': '13814000-1dd2-11b2-8080-808080808080'},
 'email': 'tenant@akiot.es',
 'first_name': None,
 'id': {'entity_type': 'USER', 'id': '87789550-ade8-11f0-b1bc-33c092a70dea'},
 'last_name': None,
 'name': 'tenant@akiot.es',
 'owner_id': {'entity_type': 'TENANT',
              'id': '86c746b0-ade8-11f0-b1bc-33c092a70dea'},
 'phone': None,
 'tenant_id': {'entity_type': 'TENANT',
               'id': '86c746b0-ade8-11f0-b1bc-33c092a70dea'},
 'version': 2}



## 3. Creamos Grupos para distintas entidades

In [3]:
# Creo un GRUPO de activos en el nivel de tenant: "farmacias_cofa"
current_user = rest_client.get_user()
owner_id = current_user.tenant_id
group_type = "ASSET"
group_name = "farmacias_cofa"

try:
    # 1.1 Intentar obtener perfiles de activos para verificar si ya existe
    # El método get_asset_profiles es paginado, por lo que buscamos en la primera página
    logging.info(f"Buscando grupo de {group_type} con nombre '{group_name}' para el propietario '{owner_id.id}'...")
    existing_group = rest_client.get_entity_group_by_owner_and_name_and_type(
            owner_id=owner_id, 
            group_type=group_type, 
            group_name=group_name
        )
    logging.info(f"El grupo de {group_type} '{group_name}' ya existe.")
    
except ApiException as e:
    if e.status == 404:
        import json
        error_body = json.loads(e.body)
        if error_body.get('message') == "Requested item wasn't found!":
            # Si se encuentra el error esperado 404:
            print("El grupo no fue encontrado (404). Puedes proceder a crearlo.")
            # Creamos el grupo
            shared_asset_group = EntityGroup(name=group_name, type=group_type)
            shared_asset_group = rest_client.save_entity_group(shared_asset_group)
            logging.info('Asset group created:\n%r\n', shared_asset_group)
        else:
            # Otro error 404 (mensaje inesperado)
            raise e # Lanza de nuevo la excepción para manejarla más arriba









2025-11-05 16:38:19 - INFO - 2330404645 - 10 - Buscando grupo de ASSET con nombre 'farmacias_cofa' para el propietario '86c746b0-ade8-11f0-b1bc-33c092a70dea'...
2025-11-05 16:38:20 - INFO - 2330404645 - 16 - El grupo de ASSET 'farmacias_cofa' ya existe.


In [4]:
# Creamos Grupo de Customers "COFA" a nivel de Tenant
current_user = rest_client.get_user()
owner_id = current_user.tenant_id
group_type = "CUSTOMER"
group_name = "COFA"

try:
    # El método get_asset_profiles es paginado, por lo que buscamos en la primera página
    logging.info(f"Buscando grupo de {group_type} con nombre '{group_name}' para el propietario '{owner_id.id}'...")
    existing_group = rest_client.get_entity_group_by_owner_and_name_and_type(
            owner_id=owner_id, 
            group_type=group_type, 
            group_name=group_name
        )
    shared_customer_group = existing_group
    logging.info(f"El grupo de {group_type} '{group_name}' ya existe.")
    
except ApiException as e:
    if e.status == 404:
        import json
        error_body = json.loads(e.body)
        if error_body.get('message') == "Requested item wasn't found!":
            # Si se encuentra el error esperado 404:
            print("El grupo no fue encontrado (404). Puedes proceder a crearlo.")
            # Creamos el grupo
            shared_customer_group = EntityGroup(name=group_name, type=group_type)
            shared_customer_group = rest_client.save_entity_group(shared_customer_group)
            logging.info('Asset group created:\n%r\n', shared_customer_group)
        else:
            # Otro error 404 (mensaje inesperado)
            raise e # Lanza de nuevo la excepción para manejarla más arriba

2025-11-05 16:38:27 - INFO - 242566287 - 9 - Buscando grupo de CUSTOMER con nombre 'COFA' para el propietario '86c746b0-ade8-11f0-b1bc-33c092a70dea'...
2025-11-05 16:38:27 - INFO - 242566287 - 16 - El grupo de CUSTOMER 'COFA' ya existe.


In [5]:
# Creo un perfil para los activos "Farmacias COFA"
from tb_rest_client.models.models_pe import EntityId
import logging
from pprint import pprint
profile_name = "Farmacias COFA"
profile_description = "Default asset profile"

# asset_profile = AssetProfile(
#         name=profile_name ,
#         description=profile_description,
#         # Otros campos opcionales:
#         # default_queue_name="Main" 
#         # default=False # No marcar como perfil por defecto
#     )

# saved_asset_profile = rest_client.save_asset_profile(asset_profile)
try:
    # 1.1 Intentar obtener perfiles de activos para verificar si ya existe
    # El método get_asset_profiles es paginado, por lo que buscamos en la primera página
    page_data = rest_client.get_asset_profiles(page_size=100, page=0, text_search=profile_name)

    existing_profile = next((profile for profile in page_data.data if profile.name == profile_name), None)

    if existing_profile:
        logging.info(f"El perfil de activo '{profile_name}' ya existe!!.")
        profile_asset_to_use = existing_profile.id
        raise Exception(f"El perfil de activo '{profile_name}' ya existe!!.")
    else:
    # 1.2 Crear una instancia del modelo AssetProfile
        new_asset_profile = AssetProfile(
                                        name=profile_name,
                                        description=profile_description,)

    # 1.3 Guardar el perfil de activo utilizando el método save_asset_profile
        created_profile = rest_client.save_asset_profile(body=new_asset_profile)

        profile_asset_to_use = created_profile.id
        logging.info(f"✅ Perfil de activo '{profile_name}' creado exitosamente.")
            
except Exception as e:
        logging.error(f"❌ Error al crear/obtener el perfil de activo: {e}")





2025-11-05 16:38:31 - INFO - 193593744 - 25 - El perfil de activo 'Farmacias COFA' ya existe!!.
2025-11-05 16:38:31 - ERROR - 193593744 - 41 - ❌ Error al crear/obtener el perfil de activo: El perfil de activo 'Farmacias COFA' ya existe!!.


## 4. Vamos ahora a loggearnos como la API CE ya que tiene implementado el acceso de atributos

In [ ]:
import logging
from json import load
# Importing models and REST client class from Professional Edition version
from tb_rest_client.rest_client_ce import *
from tb_rest_client.rest import ApiException


logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(module)s - %(lineno)d - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')


# ThingsBoard REST API URL
url = "https://development.akiot.es/"

# Default Tenant Administrator credentials
username = "tenant@akiot.es"
password = "tenant"

In [ ]:
with RestClientCE(base_url=url) as rest_client_control:
    try:
        # Auth with credentials
        rest_client_control.login(username=username, password=password)

        # Getting current user
        current_user_control = rest_client_control.get_user()
        logging.info('Current user:\n%r\n', current_user_control)

    except ApiException as e:
            logging.exception(e)

## 5. Vamos a crear el Customer con sus atributos y crear el activo FARMACIA con sus atributos y se lo vamos a asignar
Por cada activo  (Farmacia) debería Asignar:
- Un Name del activo: Farmacias Garcia 
- Un Label: Farmacias COFA
- Un Type, perfil de activo: Farmacias COFA
- Una Descripción Opcional
- Se debe crear o buscar un customer filtrado por CUIT de la farmacia, asignarle el grupo COFA. Este Customer es independiente de las x cuentas de usuario que se puedan crear para acceso.
- 
- Luego se asigna el activo a ese customer.

- Se podría crear un grupo de activo, pero en este caso no haría falta porque cada cliente tiene asignado su dispositivo

### Ejemplo de fila de datos en el CSV

| **Name**         | **Type**       | **Label**      | **cuit**       | **calleNumero**  | **ciudad** | **provincia** | **codigoPostal** | **latitud**  | **longitud** | **responsable** | **cargo** | **correo**                                            | **celular**    |
| ---------------- | -------------- | -------------- | -------------- | ---------------- | ---------- | ------------- | ---------------- | ------------ | ------------ | --------------- | --------- | ----------------------------------------------------- | -------------- |
| Farmacias Garcia | Farmacias COFA | Farmacias COFA | 20-123456789-5 | Av. Belgrano 869 | CABA       | Buenos Aires  | 1092             | -34.61241902 | -58.37865369 | Fulano Peña     | Dueño     | Fulano Peñ[a@mifarmacia.com](mailto:a@mifarmacia.com) | 549 1177000200 |






In [9]:
# Creamos, el customer y lo asociamos a su grupo "COFA", luego creamos el activo para cada fila del CSV, luego establecemos la relación entre ambos.
import csv

file_path = 'activos_COFA_ejemplo.csv'

try:
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        lector = csv.DictReader(csvfile)  # Usa la primera fila como encabezados

        for fila in lector:
            try:
                # Cada fila es un diccionario
                datos = fila

                # Acceder a los valores por nombre de columna
                name = datos.get("Name", "")
                tipo = datos.get("Type", "")
                label = datos.get("Label", "")
                cuit = datos.get("cuit_farmacia", "")
                calleNumero = datos.get("calleNumero", "")
                ciudad = datos.get("ciudad", "")
                provincia = datos.get("provincia", "")
                codigoPostal = datos.get("codigoPostal", "")
                latitud = datos.get("latitud", "")
                longitud = datos.get("longitud", "")
                responsable = datos.get("responsable", "")
                cargo = datos.get("cargo", "")
                correo = datos.get("correo", "")
                celular = datos.get("celular", "")
                chat_id = datos.get("chat_id", "")
                token_telegram = datos.get("token_telegram", "")

                # Intentamos crear el customers
                try:
                    # 1.1 Intentar obtener todos los customers para verificar si ya existe
                    # page_data = rest_client.get_customers(page_size=100, page=0, text_search="CP1097")
                    page_data = rest_client.get_customers(page_size=1000, page=0)

                    # existing_profile_customer = next((profile for profile in page_data.data if profile.address2  == cuit), None)
                    existing_profile_customer = next((profile for profile in page_data.data if profile.email == correo), None)

                    if existing_profile_customer:
                        # logging.info(f"El customer {responsable} ya existe !!, el CUIT asociado es {cuit} y su correo es  {correo} ")

                        # SERIA MEJOR FILTRAR POR DNI DEL RESPONSABLE, SOLICITAR QUE LO AGREGUEN AL CSV
                        logging.info(f"El customer {responsable} ya existe !!, su correo es  {correo} ")
                        # Tomando los datos de la fila n, acá creo el activo y se lo asigno a ese customer ya existente
                        # Ver si el activo existe por cuit, tal que el cuit puede ser la etiqueta o un atributo del servidor.
                        # Luego se debe asignar el perfil creado para el activo Farmacias COFA
                        # En este caso se supone que el customer ya tiene sus atributos creados correctamente y solo se corre aquí para asignarle nuevos activos.
                                               
                        
                         # Creating an Asset
                         
                        asset = Asset(name=name, label=cuit,
                                    asset_profile_id=profile_asset_to_use,
                                    customer_id=existing_profile_customer.id)
                        asset = rest_client.save_asset(asset)

                        # Cargo los atributos de servidor del ASSET Farmacia
                        body_to_attributes = {
                            "correo": correo,
                            "dirección": calleNumero + ", " + ciudad + ", " + provincia + ", " + codigoPostal,
                            # "imagenEdificio": imagenEdificio,
                            "latitud": latitud,
                            "longitud": longitud,
                            "personaContacto": responsable,
                            "telefono": celular,
                            "cuit_farmacia": cuit

                        }

                        # Guardo los atributos de servidor para este asset
                        try:
                            result = rest_client.telemetry_controller.save_entity_attributes_v1_using_post(
                                entity_type="ASSET",
                                entity_id=asset.id,
                                scope="SERVER_SCOPE",
                                body=body_to_attributes
                            )
                            print("✅ Atributo guardado correctamente:", result)
                        except ApiException as e:
                            print("❌ Error al guardar atributo:", e)

                        logging.info("Asset was created:\n%r\n", asset)


                        raise Exception(f"El perfil de activo '{profile_name}' ya existe!!.")
                    else:
                    # 1.2 Creamos el customer
                        customer1 = Customer(title=responsable,
                                            email=correo,
                                            country="Argentina", # Valor fijo
                                            state=provincia,
                                            city=ciudad,
                                            address=calleNumero,
                                            address2=cuit,  # Uso address2 para guardar el CUIT de la Farmacia y buscar luego
                                            zip=codigoPostal,
                                            phone=celular,)
                                            
                        customer1 = rest_client.save_customer(body=customer1)
                        # Asocio el customer al grupo "COFA"
                        rest_client.add_entities_to_entity_group(shared_customer_group.id, [customer1.id.id])
                        logging.info(f'✅ Customer creado exitosamente: {customer1.title}')
                        # Creo los atributos en el customer
                        # # Veo los atributos del customer
                        # atributos_customer = rest_client.telemetry_controller.get_attributes_by_scope_using_get(
                        #                                                                                         entity_type="CUSTOMER",
                        #                                                                                         entity_id=existing_profile_customer.id,
                        #                                                                                         scope="SERVER_SCOPE",
                        #                                                                                         )

                        # Cargo nuevos atributos al customer
                        body_to_attributes = {
                            "cuit_farmacia": cuit,
                            "chat_id": chat_id,
                            "token_telegram": token_telegram,
                            "email": correo

                        }
                        # Guardo un nuevo atributo de servidor para este customer
                        try:
                            result = rest_client.telemetry_controller.save_entity_attributes_v1_using_post(
                                entity_type="CUSTOMER",
                                entity_id=customer1.id,
                                scope="SERVER_SCOPE",
                                body=body_to_attributes
                            )
                            print("✅ Atributo guardado correctamente:", result)
                        except ApiException as e:
                            print("❌ Error al guardar atributo:", e)
                        
                        # Creating an Asset
                         
                        asset = Asset(name=name, label=cuit,
                                    asset_profile_id=profile_asset_to_use,
                                    customer_id=customer1.id)
                        asset = rest_client.save_asset(asset)

                        # Cargo los atributos de servidor del ASSET Farmacia
                        body_to_attributes = {
                            "correo": correo,
                            "dirección": calleNumero + ", " + ciudad + ", " + provincia + ", " + codigoPostal,
                            # "imagenEdificio": imagenEdificio,
                            "latitud": latitud,
                            "longitud": longitud,
                            "personaContacto": responsable,
                            "telefono": celular,
                            "cuit_farmacia": cuit

                        }

                        # Guardo los atributos de servidor para este asset
                        try:
                            result = rest_client.telemetry_controller.save_entity_attributes_v1_using_post(
                                entity_type="ASSET",
                                entity_id=asset.id,
                                scope="SERVER_SCOPE",
                                body=body_to_attributes
                            )
                            print("✅ Atributo guardado correctamente:", result)
                        except ApiException as e:
                            print("❌ Error al guardar atributo:", e)

                        logging.info("Asset was created:\n%r\n", asset)
                        # Tomando los datos de la fila n, acá creo el activo y se lo asigno a ese customer recientemente creado
                except Exception as e:
                        logging.error(f"Error al intentar crear el customer: {e}")
                

            except Exception as e:
                print(f"⚠️ Error procesando la fila: {fila}")
                print(f"   Detalle: {e}")

except FileNotFoundError:
    print("❌ No se encontró el archivo CSV.")
except Exception as e:
    print(f"❌ Error general al leer el archivo: {e}")


2025-11-05 16:42:45 - INFO - 2172448590 - 104 - ✅ Customer creado exitosamente: Fulano Peña


✅ Atributo guardado correctamente: b''
✅ Atributo guardado correctamente: b''


2025-11-05 16:43:43 - INFO - 2172448590 - 165 - Asset was created:
{'additional_info': None,
 'asset_profile_id': {'entity_type': 'ASSET_PROFILE',
                      'id': '92ecdd90-ba42-11f0-9661-1da9e45f2bbc'},
 'created_time': 1762360999738,
 'customer_id': {'entity_type': 'CUSTOMER',
                 'id': '7208df60-ba66-11f0-9661-1da9e45f2bbc'},
 'id': {'entity_type': 'ASSET', 'id': '890f09a0-ba66-11f0-9661-1da9e45f2bbc'},
 'label': '20-44267315-5',
 'name': 'Famacia DEMO COFA',
 'tenant_id': {'entity_type': 'TENANT',
               'id': '86c746b0-ade8-11f0-b1bc-33c092a70dea'},
 'type': 'Farmacias COFA',
 'version': 1}

2025-11-05 16:44:50 - INFO - 2172448590 - 104 - ✅ Customer creado exitosamente: Juan Corvalan


✅ Atributo guardado correctamente: b''
✅ Atributo guardado correctamente: b''


2025-11-05 16:44:57 - INFO - 2172448590 - 165 - Asset was created:
{'additional_info': None,
 'asset_profile_id': {'entity_type': 'ASSET_PROFILE',
                      'id': '92ecdd90-ba42-11f0-9661-1da9e45f2bbc'},
 'created_time': 1762361090546,
 'customer_id': {'entity_type': 'CUSTOMER',
                 'id': 'bb88ca60-ba66-11f0-9661-1da9e45f2bbc'},
 'id': {'entity_type': 'ASSET', 'id': 'bf2f3d20-ba66-11f0-9661-1da9e45f2bbc'},
 'label': '20-45445464-5',
 'name': 'Famacia TEST',
 'tenant_id': {'entity_type': 'TENANT',
               'id': '86c746b0-ade8-11f0-b1bc-33c092a70dea'},
 'type': 'Farmacias COFA',
 'version': 1}

2025-11-05 16:44:57 - INFO - 2172448590 - 46 - El customer Fulano Peña ya existe !!, su correo es  fpenia@akribis.info 


✅ Atributo guardado correctamente: b''


2025-11-05 16:45:47 - INFO - 2172448590 - 85 - Asset was created:
{'additional_info': None,
 'asset_profile_id': {'entity_type': 'ASSET_PROFILE',
                      'id': '92ecdd90-ba42-11f0-9661-1da9e45f2bbc'},
 'created_time': 1762361129307,
 'customer_id': {'entity_type': 'CUSTOMER',
                 'id': '7208df60-ba66-11f0-9661-1da9e45f2bbc'},
 'id': {'entity_type': 'ASSET', 'id': 'd649b2b0-ba66-11f0-9661-1da9e45f2bbc'},
 'label': '30-71234567-8',
 'name': 'Farmacia CENTRO COFA 2',
 'tenant_id': {'entity_type': 'TENANT',
               'id': '86c746b0-ade8-11f0-b1bc-33c092a70dea'},
 'type': 'Farmacias COFA',
 'version': 1}

2025-11-05 16:45:51 - ERROR - 2172448590 - 168 - Error al intentar crear el customer: El perfil de activo 'Farmacias COFA' ya existe!!.
